### Import Libraries

In [5]:
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
import pandas as pd
import numpy as np
import os

### Web Scraping - Selenium

In [ ]:
# Define o diretório raiz da aplicação
cwd = os.getcwd()

# Determinar opções do chrome para definir o diretório de download de arquivos
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : cwd}

# Setar as opções (opções de tamanho da tela, posição do navegador, diretório para download, etc.)
chromeOptions.add_experimental_option("prefs",prefs)

# Setar o diretório onde se encontra o chromedriver. Para utilizar o Selenium é necessário um driver para
# fazer a interface com o navegador escolhido
chromedriver = "C:/Users/Leonardo/Desktop/palestra pypoa/chromedriver.exe"

# Instanciar o driver do Chrome com as opções definidas
driver = webdriver.Chrome(executable_path=chromedriver, chrome_options=chromeOptions)

# URL da página do repositório Lume da UFRGS
driver.get("http://www.lume.ufrgs.br/handle/10183/15757")


enviar = driver.find_element_by_id('aspect_discovery_CommunityCollectionSearchFilter_field_submit-search')
enviar.submit()
trabalhos = np.arange(0,10)
n_paginas = np.arange(1,2415)
pagina = []
data = pd.DataFrame()

# Lista com a URL de todas as páginas
for h in n_paginas:
    pagina.append("https://lume.ufrgs.br/handle/10183/15757/discover?rpp=10&etal=0&group_by=none&page="+str(h)+"&querytype_0=title&query_relational_operator_0=contains&query_value_0=&querytype_1=authortd&query_relational_operator_1=contains&query_value_1=&querytype_2=orientador&query_relational_operator_2=contains&query_value_2=&querytype_3=curso&query_relational_operator_3=equals&query_value_3=&querytype_4=subject&query_relational_operator_4=contains&query_value_4=&querytype_6=dataAno&query_relational_operator_6=equals&query_value_6=&querytype_7=idioma&query_relational_operator_7=equals&query_value_7=&querytype_8=formatoArquivo&query_relational_operator_8=equals&query_value_8=&query=")

# Loop de iteração por página
for h in pagina:
    driver.get(h)
    div = driver.find_element_by_id('aspect_discovery_SimpleSearch_div_search-results')
    
    # Captura a URL dos 10 trabalhos contidos na página (links diplicados)
    link =[]
    for value in div.find_elements_by_css_selector("a"): # os valores do elemento são acessados por iteração
        link.append(value.get_attribute('href'))
    
    # Remove links duplicados
    item = []
    for value in np.arange(0,len(link),2):    
        try:
            item.append(link[value])
        except:
            pass
    link = item
    
    # Loop de iteração para coletar informação de cada um dos 10 trabalhos contidos em cada página
    for i in trabalhos:
        
        # Acessa a URL do trabalho
        driver.get(link[i]+"?show=full")
        
        # Captura a informação da tabela que contém os dados de interesse
        table = driver.find_element_by_class_name('ds-includeSet-table')
        
        # Captura a primeira coluna com o nome das variáveis
        header =[]
        for value in table.find_elements_by_class_name("label-cell"):
            header.append(value.text)
        
        # Captura os dados de toda a tabela
        rows =[]
        for value in table.find_elements_by_tag_name("td"):
            rows.append(value.text)
        
        # Remover o header dos dados que compõem as linhas
        for value in header:
            rows.remove(value)
            
        # Concatenar a coluna 2 com a coluna 3 da tabela e '@;' entre elas
        item = []
        for value in np.arange(0,len(rows),2):
            try:
                item.append(rows[value]+'@;'+rows[value+1])
            except:
                pass
        rows = item
        
        # Tratamento especial para o resumo, pois pode ter mais de um. Todos são concatenados com '&;'
        abstract = []
        for i in range(len(header)):
            if str(header[i]).find("abstract") != -1:
                abstract.append(i)         
        a = ""
        for i in (abstract):
            a = rows[i] + "&;" + a
            
        # Tratamento especial para o autor, pois pode ter mais de um. Todos são concatenados com '&;'            
        author = []
        for i in range(len(header)):
            if str(header[i]).find("author") != -1:
                author.append(i)             
        b = ""
        for i in (author):
            b = rows[i] + "&;" + b
         
        # Criar o data frame no pandas
        rows = pd.DataFrame(rows).T
        rows.columns = header

        # Deletar colunas duplicadas para cria-las novamente depois (evitar erro no append do pandas)
        try:
            rows = rows.drop(columns = [header[abstract[0]]])
        except:
            pass
        try:
            rows = rows.drop(columns = [header[author[0]]])
        except:
            pass
        
        # Informaçao da plataforma que será armazenada para cada trabalho
        columns_keep = ["dc.contributor.author","dc.date.issued","dc.identifier.uri","dc.title","dc.degree.graduation","dc.description.abstract","dc.degree.level"]
        rows['dc.description.abstract'] = a
        rows['dc.contributor.author'] = b
        rows = rows[columns_keep]

        # Popular o o data frame com os dados coletados
        if data.empty:
            data = pd.DataFrame(columns = ["dc.contributor.author","dc.date.issued","dc.identifier.uri","dc.description.abstract","dc.title","dc.degree.graduation","dc.degree.level"])
        data = data.append(rows,ignore_index=True)
        
        # Voltar para a página anterior e repetir o processo
        driver.back()
        
# Salvar os dados coletados em .xlsx
data.to_excel('dados_ufrgs_.xlsx', index = False)